## Sistema Multi-Agente per lo Sviluppo Software con LLM

Questo notebook implementa un **sistema multi-agente** basato su **CrewAI** e integrato con **OpenRouter.AI**, in cui più **modelli linguistici di grandi dimensioni (LLM)** collaborano in modo coordinato, ciascuno con un **ruolo specifico** all'interno del ciclo di vita di un progetto software.  
L’obiettivo è simulare un team di sviluppo virtuale in grado di gestire un progetto software dall'analisi iniziale fino alla verifica finale.

---

### 👥 Ruoli e Responsabilità

- **Project Manager**
- **Solution Architect**
- **Technical Lead**
- **Frontend Developer**
- **Backend Developer**
- **Database Administrator**
- **QA & Test Engineer**

---

### 🔁 Flusso del Processo Collaborativo

```text
Project Manager → Solution Architect → Technical Lead
                              ↓
       Frontend Dev ←→ Backend Dev ←→ DB Admin
                              ↓
                     QA & Test Engineer
```

---

### 🧭 Descrizione del Processo

- Il **Project Manager** riceve in input i parametri utente (`project_name`, `project_type`, `project_requirements`) e redige un documento di *Specifiche di Progetto*.
- Il **Solution Architect** definisce l’architettura tecnica del sistema, selezionando i componenti principali e le linee guida progettuali.
- Il **Technical Lead** costruisce un *piano di implementazione* articolato in un backlog di attività, assegnando priorità, responsabili e dipendenze.
- Gli sviluppatori **Frontend**, **Backend** e il **Database Administrator** realizzano le varie componenti applicative seguendo il piano stabilito.
- Il **QA & Test Engineer** verifica la correttezza, la robustezza e la coerenza delle implementazioni rispetto ai requisiti iniziali e alle specifiche tecniche.

I deliverables prodotti da ogni agent vengono salvati in formato MarkDown nella cartella /outputs

In [ ]:
import os
import yaml
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

# Caricamento variabili d'ambiente
load_dotenv("vars.env")

In [ ]:
# Parametri del progetto
project_name = "ToDo list per la gestione di attività personali"
project_type = "Web App"
project_requirements = [
    # Requisiti Funzionali
    "L'utente può creare nuove attività con titolo, descrizione, data di scadenza e priorità",
    "Le attività devono essere visualizzabili in una lista ordinabile per data, priorità o stato",
    "L'utente può modificare attività esistenti (titolo, descrizione, scadenza, priorità)",
    "L'utente può eliminare attività dalla lista",
    "L'utente può contrassegnare un'attività come completata",
    "È possibile filtrare le attività per stato (completate/in sospeso), priorità e cercarle per testo",
    "Il sistema può inviare notifiche per attività prossime alla scadenza",
    "Le attività possono essere assegnate a categorie personalizzate o etichette (tag)",
    "Il software consente di salvare e ripristinare le attività (backup locale o cloud)",
    "Le attività si sincronizzano tra più dispositivi con lo stesso account",
   
    # Requisiti Non Funzionali
    "Interfaccia semplice, intuitiva e usabile anche da utenti non esperti",
    "Tempi di risposta rapidi anche con molte attività",
    "Supporto per più piattaforme (desktop, mobile, web)",
    "Il sistema garantisce l'integrità e la persistenza dei dati inseriti",
    "I dati dell'utente devono essere protetti, soprattutto se salvati nel cloud",
    "Il software deve essere scalabile per l'aggiunta futura di nuove funzionalità",

    # Requisiti Tecnici
    "Supporto per frontend in React, Angular, Vue o Flutter",
    "Utilizzo di backend in Node.js, Python (Django/Flask) o Java (Spring Boot)",
    "Persistenza dati locale (SQLite) o remota tramite API REST e database relazionale",
    "Architettura del software basata su MVC o MVVM",
    "Integrazione con notifiche push (Firebase, OneSignal) e autenticazione (OAuth, Google Sign-In)",
    "Presenza di test unitari e di integrazione per garantire la qualità del software"       
]

In [ ]:
def load_agents(path):
    with open(path, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f)
    agents = {}
    
    for key, cfg in data.items():
        # Usa il modello specificato nel YAML o gpt-3.5-turbo come default
         # Configurazione di default per llm_config
        default_llm_config = {
            "model": "openai/gpt-3.5-turbo",
            "temperature": 0.7,
            "openai_api_key": os.getenv("OPENAI_API_KEY"),
            "base_url": os.getenv("OPENAI_API_BASE")
        }
        
        llm_config = cfg.get("llm_config", default_llm_config)
        
        # Creazione dell'istanza ChatOpenAI    
        llm = ChatOpenAI(
            openai_api_key=os.getenv("OPENAI_API_KEY"),
            base_url=os.getenv("OPENAI_API_BASE"),
            llm_config=llm_config
        )
        
        agent = Agent(
            role=cfg["role"],
            goal=cfg["goal"],
            backstory=cfg["backstory"],
            verbose=cfg.get("verbose", False),
            allow_delegation=cfg.get("allow_delegation", False),
            llm=llm
        )
        agents[key] = agent
    return agents

def load_tasks(path, agents_dict):
    with open(path, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f)

    task_objs = {}
    for key, cfg in data.items():
        agent = agents_dict[cfg["agent"]]
        task = Task(
            description=cfg["description"],
            expected_output=cfg.get("expected_output", ""),
            agent=agent
        )
        # Aggiungiamo output_file come attributo custom
        task.output_file = cfg.get("output_file", f"outputs/{key}.md")
        task_objs[key] = task

    for key, cfg in data.items():
        if "context" in cfg:
            task_objs[key].context = [task_objs[cid] for cid in cfg["context"]]

    return list(task_objs.values())

In [ ]:
agents = load_agents("agents.yaml")
tasks = load_tasks("tasks.yaml", agents)

In [ ]:
crew = Crew(
    agents=list(agents.values()),
    tasks=tasks,
    verbose=False
)

In [ ]:
# The given Python dictionary
inputs = {
  'project_name': project_name,
  'project_type': project_type,
  'project_requirements': project_requirements
}

In [ ]:
# Run the crew
result = crew.kickoff(
  inputs=inputs
)

In [ ]:
def save_task_outputs(tasks):
    for task in tasks:
        output = task.output or ""
        # Se il task ha attributo .output_file, salviamo
        if hasattr(task, "output_file"):
            filepath = task.output_file
        else:
            # Se non c'è, proviamo da description o role
            name = task.agent.role.lower().replace(" ", "_")
            filepath = f"outputs/{name}_output.md"
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(output.content if hasattr(output, "content") else str(output))

save_task_outputs(tasks)